In [1]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import pickle
import itertools
import os
import psutil
import gc
import random
import pynvml

random.seed(42)
np.random.seed(42)

In [2]:
# Function to clean folder names
def clean_folder_name(folder_name):
    # Remove invalid characters
    cleaned_name = re.sub(r'[<>:"/\\|?*]', '', folder_name)
    # Remove trailing dots and spaces
    cleaned_name = cleaned_name.rstrip('. ')
    return cleaned_name


def CPU_monitor_memory_usage():
    memory_info = psutil.virtual_memory()
    memory_usage = memory_info.percent
        
    print(f"CPU Current memory usage: {memory_usage}%")

    if memory_usage >= 95:
        print("CPU Memory usage is too high. Pausing execution...")
        gc.collect()  # Trigger garbage collection manually
        while memory_usage > 30:
            time.sleep(10)
            memory_info = psutil.virtual_memory()
            memory_usage = memory_info.percent
        print("CPU Memory usage is low enough. Resuming execution...")

    # time.sleep(5)

def monitor_gpu_memory():
    # Initialize NVML
    pynvml.nvmlInit()
    
    try:
        # Get handle for the first GPU
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)

        # Get memory info
        mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        total_memory = mem_info.total
        used_memory = mem_info.used

        # Calculate the percentage of GPU memory used
        memory_usage = (used_memory / total_memory) * 100
        print(f"Current GPU memory usage: {memory_usage:.2f}%")

        # Check if memory usage is too high
        if memory_usage >= 95:
            print("GPU memory usage is too high. Pausing execution...")
            while memory_usage > 30:
                time.sleep(10)
                mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
                used_memory = mem_info.used
                memory_usage = (used_memory / total_memory) * 100
            print("GPU memory usage is low enough. Resuming execution...")

    finally:
        # Clean up
        pynvml.nvmlShutdown()

def dict_to_foldername(param_dict, max_values=2):
    # Flatten the dictionary into a string of 'key=abbreviated_values' pairs
    parts = []
    for key, values in param_dict.items():
        # Abbreviate key names
        short_key = ''.join(word[0] for word in key.split('_'))
        
        # Convert list of values to a comma-separated string
        value_str = ','.join(map(str, values))
        # Form key=value string and append to the parts list
        parts.append(f"{short_key}={value_str}")

    # Join all parts with a separator and prepend prefix
    folder_name = "_".join(parts)

    # Replace any potentially problematic characters (if any)
    folder_name = folder_name.replace(":", "-").replace("/", "-").replace("\\", "-")

    return folder_name

def make_hashable(dict_obj):
    return tuple(sorted(dict_obj.items()))

In [3]:
def CatList(cat_, list_, type_ = "and"):
        if type_ == "and":
            return cat_ in list_
        elif type_ == "or":
            return cat_ not in list_
        else:
            return True
def cats_used(max_cat_others, data, output_cat, min_cats = 2):
    if max_cat_others > 0:
        cats_values = (data[output_cat].value_counts()/data.shape[0])*100
        sum_perc = 0
        cats_ = []
        for i in range(len(cats_values)):
            if len(cats_) < min_cats or sum_perc < (100-max_cat_others):
                cats_.append(str(cats_values.index[i]))
                sum_perc = sum_perc + cats_values[i]
            else:
                break

    else:
        cats_ = list(pd.unique(data[output_cat].tolist()))

    return cats_

def cats_levels(data, max_cat_others, output_cat, min_cats,prev_cats= {}):
    cats_to_use = []
    if len(list(prev_cats.keys())) != 0:
        for cat_name in prev_cats.keys():
            print(cat_name)
            for case in prev_cats[cat_name]:
                print(case)
                data_cat = data[data[cat_name] == case]
                cats_to_use_i = cats_used(max_cat_others, data_cat, output_cat, min_cats)
                cats_to_use = cats_to_use + cats_to_use_i
    else:
        cats_to_use = cats_used(max_cat_others, data, output_cat, min_cats)

    return cats_to_use


def Cats_Filter(instance, list_categories, name_prev_cat = None):
    if name_prev_cat != None:
        if instance in list_categories:
            return instance
        else:
            return "Other_TEIS_"+name_prev_cat
    else:
        if instance in list_categories:
            return instance
        else:
            return "Other_TEIS"

def ManageTextFeature(text_):
    if type(text_) != str:
        return "No valid text"
    else:
        return text_


def classification_report_to_df(report, y_true, y_pred):
    global bch_class_df
    global topic_dict
    global iteration
    df = pd.DataFrame(report).transpose()

    order_labels = list(topic_dict.values())

    # Calculate the confusion matrix
    labels = df.index[:-3]  # Exclude 'accuracy', 'macro avg', 'weighted avg'
    # Calculate the confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    # Extracting TP, FP, TN, FN for each class
    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (FP + FN + TP)

    sens = sum(TP) / (sum(TP)+sum(FN))
    spec = sum(TN) / (sum(TN)+sum(FP))
    
    # Calculate Sensitivity (same as recall)
    df['Sensitivity'] = df['recall']
    
    # Calculate Specificity
    tn = cm.sum() - (cm.sum(axis=0) + cm.sum(axis=1) - np.diag(cm))
    fp = cm.sum(axis=0) - np.diag(cm)
    specificity = tn / (tn + fp)
    
    # Assign computed specificity to dataframe except for the last three rows
    df.loc[df.index[:-3], 'Specificity'] = specificity
    
    # Handling special cases
    # Set 'accuracy' row sensitivity and specificity to the accuracy value
    accuracy = df.loc['accuracy', 'precision']  # assuming 'precision' contains the accuracy
    df.loc['accuracy', ['Sensitivity', 'Specificity']] = sens, spec
    
    # Calculate 'macro avg' and 'weighted avg' for sensitivity and specificity
    df.loc['macro avg', 'Sensitivity'] = df.iloc[:-3]['Sensitivity'].mean()
    df.loc['weighted avg', 'Sensitivity'] = np.average(df.iloc[:-3]['Sensitivity'], weights=df.iloc[:-3]['support'])
    
    df.loc['macro avg', 'Specificity'] = df.iloc[:-3]['Specificity'].mean()
    df.loc['weighted avg', 'Specificity'] = np.average(df.iloc[:-3]['Specificity'], weights=df.iloc[:-3]['support'])

    # Calculate Balanced Accuracy for each row, including special averages
    df['Balanced Accuracy'] = (df['Sensitivity'] + df['Specificity']) / 2

    df.loc['accuracy', 'precision'] = sum(TP) / (sum(TP) + sum(FP))
    df.loc['accuracy', 'recall'] = sum(TP) / (sum(TP) + sum(FN))
    df.loc['accuracy', 'f1-score'] = 2* sum(TP) / (2 * sum(TP) + sum(FP) + sum(FN))

    if iteration > 1:
        bch_class_df_noFr = bch_class_df.drop(columns=['TP', 'FP', 'TN', 'FN'])
    else: 
        bch_class_df_noFr = bch_class_df

    diff_df = df - bch_class_df_noFr
    # Renaming columns for clarity
    diff_df.columns = ['Diff ' + col for col in diff_df.columns]

    # Concatenating the original dataframe with the differences
    combined_df = pd.concat([df, diff_df], axis=1)

    class_accuracy = cm.diagonal() / cm.sum(axis=1)
    combined_df.loc[labels, 'Accuracy'] = class_accuracy
    # Copying f1-score to 'Accuracy' for the last three rows
    combined_df.loc[['accuracy', 'macro avg', 'weighted avg'], 'Accuracy'] = combined_df.loc[['accuracy', 'macro avg', 'weighted avg'], 'f1-score']

    # Calculate and append TP, FP, TN, FN metrics
    metrics_df = pd.DataFrame({
        "TP": TP,
        "FP": FP,
        "TN": TN,
        "FN": FN
    }, index=labels)

    # Merge the new metrics into the existing DataFrame
    combined_df = combined_df.merge(metrics_df, left_index=True, right_index=True, how='left')
    if 'accuracy' in order_labels:
        # Reorder DataFrame based on specified order labels
        combined_df = combined_df.reindex(order_labels + ['macro avg', 'weighted avg'])
    else:
        combined_df = combined_df.reindex(order_labels + ['accuracy', 'macro avg', 'weighted avg'])

    return combined_df


def dominates(score1, score2):
    return (score1[0] > score2[0] and score1[1] >= score2[1]) or (score1[0] >= score2[0] and score1[1] > score2[1])

def observer_function(all_segments_dict, pca_pair):
    global NSW_results_path
    global PCA_stats_df_name
    global PCA_stats_df
    global topic_name
    global topic_number
    global history_pareto_segments_list
    global fold_results_df
    global NSW_results_name
    global used_pca_pairs
    global bch_m0

    for i, (key, seg) in enumerate(all_segments_dict.items()):
        classification_df = seg['classification_df']

        new_row_index = len(fold_results_df)
        class_DF_path = f'{NSW_results_path}/Class_DF'
        os.makedirs(class_DF_path, exist_ok=True)
        classification_df.to_csv(f'{class_DF_path}/{topic_number}_NSW_{new_row_index}_AllEval_ClassDF.csv', index=True)
        classification_df.to_pickle(f'{class_DF_path}/{topic_number}_NSW_{new_row_index}_AllEval_ClassDF.pkl')

        # Collect all generation data into a new DataFrame row
        new_ParamCV_row = {
            "topic_name": topic_name,
            "topic_number": topic_number,
            "PCA_index": used_pca_pairs.index(seg['PCA']),
            "PCA": seg['PCA'],
            "coordinates": seg['coordinates'],
            'fitness_score': seg['fitness_score'],
            "accuracy": seg['fitness_score'][0],
            "topic_recall": seg['fitness_score'][1],
            'balanced_fitness_score': (classification_df.loc['accuracy', 'Balanced Accuracy'], classification_df.loc[topic_name, 'Balanced Accuracy']),
            'overall_balanced_accuracy': classification_df.loc['accuracy', 'Balanced Accuracy'],
            'topic_balanced_accuracy': classification_df.loc[topic_name, 'Balanced Accuracy'],
            'balanced_acc_rec_score': (classification_df.loc[topic_name, 'Balanced Accuracy'], classification_df.loc[topic_name, 'recall']),
            'topic_F1': classification_df.loc[topic_name, 'f1-score'],
            'overall_F1': classification_df.loc['accuracy', 'f1-score'],
            'overall_recall': classification_df.loc['accuracy', 'recall'],
            "retraining_time": seg["retraining_time"],
            "number_of_syn_sample": seg['number_of_syn_sample'],
            "retrained_dots_list": seg['retrained_dots_list'],
            'true_labels': seg['true_labels'],
            'predicted_labels': seg['predicted_labels'],
            "segment_key": key,
            'classDF_path': f'{class_DF_path}/{topic_number}_NSW_{new_row_index}_AllEval_ClassDF.csv',
            'T13_TBA_Imp': classification_df.loc['Humanitarian aid for Ukraine.', 'Balanced Accuracy'] - bch_m0.loc['Humanitarian aid for Ukraine.', 'Balanced Accuracy']
        }
        # # Transform DataFrame to dict format
        # for idx in classification_df.index:
        #     if idx != 'accuracy' and idx != 'macro avg' and idx != 'weighted avg':
        #         new_ParamCV_row[idx] = classification_df.loc[idx].dropna().to_dict()
    

        # Convert the dictionary to a DataFrame for a single row
        new_ParamCV_row_df = pd.DataFrame([new_ParamCV_row])
        # Concatenate this new row DataFrame to the existing DataFrame
        fold_results_df = pd.concat([fold_results_df, new_ParamCV_row_df], ignore_index=True)
        fold_results_df.to_csv(f'{NSW_results_path}/NSWAllSegs_{NSW_results_name}.csv', index=False)
        fold_results_df.to_pickle(f'{NSW_results_path}/NSWAllSegs_{NSW_results_name}.pkl')

    objective_1_values = [seg['fitness_score'][0] for seg in all_segments_dict.values() if seg['fitness_score'][0] is not None]
    objective_2_values = [seg['fitness_score'][1] for seg in all_segments_dict.values() if seg['fitness_score'][1] is not None]

    # Check if both lists are empty or filled with zeros
    if not objective_1_values or not objective_2_values or all(value == 0 for value in objective_1_values + objective_2_values):
        print("All objectives are None or 0, skipping further processes.")
        return  # Exit the function

    non_dominated_segments = {}
    for key1, dict1 in all_segments_dict.items():
        if dict1['fitness_score'] == (None, None):
            continue  # Skip non-evaluable segments

        dominated = False
        for key2, dict2 in all_segments_dict.items():
            if key1 != key2 and dominates(dict2['fitness_score'], dict1['fitness_score']):
                dominated = True
                break
        if not dominated:
            non_dominated_segments[key1] = dict1

    pareto_fitness_tuples = [(ft[0], ft[1]) for ft in [seg['fitness_score'] for seg in non_dominated_segments.values()] if ft[0] is not None and ft[1] is not None]
    pareto_segments_tuples = [(seg['coordinates'], seg['number_of_syn_sample'], seg['retrained_dots_list']) for seg in non_dominated_segments.values()]

    # Calculate statistics
    worst_fitness = (min(objective_1_values), min(objective_2_values))
    best_fitness = (max(objective_1_values), max(objective_2_values))
    mean_OverallAcc = np.mean(objective_1_values)
    mean_ClassRecall = np.mean(objective_2_values)

    print(f"PCA: {pca_pair}, Evaluations: {len(all_segments_dict)}")
    print(f"Best Fitness: {best_fitness}")
    print(f"Worst Fitness: {worst_fitness}")
    print(f"Mean Overall Accuracy: {mean_OverallAcc}")
    print(f"Mean {topic_name} Recall: {mean_ClassRecall}")

    print("Pareto Front Selections:---------------------")
    for i, (key, seg) in enumerate(non_dominated_segments.items()):
        print(f"Segment {i+1}: {seg['coordinates']}, Num Samples {seg['number_of_syn_sample']}: {seg['retrained_dots_list']} \nFitness: {seg['fitness_score']}")
        history_pareto_segments_list.append(key)
        print('---')
    print('------------------------')

    # Plotting
    plt.figure(figsize=(12, 7))
    plt.scatter(objective_1_values, objective_2_values, c='blue', alpha=0.5, label='Population')
    if pareto_fitness_tuples:
        plt.scatter([ft[0] for ft in pareto_fitness_tuples], [ft[1] for ft in pareto_fitness_tuples], c='red', alpha=0.9, label='Pareto Front')
    
    plt.xlabel('Objective 1: Overall Accuracy')
    plt.ylabel(f'Objective 2: {topic_name} Recall')
    plt.title(f'{topic_number}, Population and Pareto Front at PCA {pca_pair}')
    plt.legend()
    pareto_plots_dir = f"{NSW_results_path}/{PCA_stats_df_name}"
    os.makedirs(pareto_plots_dir, exist_ok=True)
    filename = f"{pca_pair[0]}{pca_pair[1]}.png"
    plt.savefig(os.path.join(pareto_plots_dir, filename), dpi=200, bbox_inches='tight', pad_inches=0)
    plt.show()

    recall_key = f"Mean {topic_name} Recall"
    # Collect all generation data into a new DataFrame row
    new_row = {
        "PCA": f"{pca_pair[0]}{pca_pair[1]}",
        "Number of Evaluations": len(all_segments_dict),
        "Best Fitness": best_fitness,
        "Worst Fitness": worst_fitness,
        "Mean Overall Accuracy": mean_OverallAcc,
        recall_key: mean_ClassRecall,
        "Pareto Front Segments": [pareto_segments_tuples, pareto_fitness_tuples]
    }
    # Convert the dictionary to a DataFrame for a single row
    new_row_df = pd.DataFrame([new_row])
    # Concatenate this new row DataFrame to the existing DataFrame
    PCA_stats_df = pd.concat([PCA_stats_df, new_row_df], ignore_index=True)
    PCA_stats_df.to_csv(f'{NSW_results_path}/{PCA_stats_df_name}/{pca_pair[0]}{pca_pair[1]}.csv', index=False)
    PCA_stats_df.to_pickle(f'{NSW_results_path}/{PCA_stats_df_name}/{pca_pair[0]}{pca_pair[1]}.pkl')


def final_pareto_observer(history_pareto_segments_list):
    global history_segments_dict_allPCA
    global topic_name
    global topic_number
    global NSW_results_path
    global PCA_stats_df_name
    global level_stats_df
    global fold_pfs_df
    global NSW_results_name
    global used_pca_pairs
    global PCA_stats_df

    objective_1_values = [history_segments_dict_allPCA[seg_key]['fitness_score'][0] for seg_key in history_pareto_segments_list if history_segments_dict_allPCA[seg_key]['fitness_score'][0] is not None]
    objective_2_values = [history_segments_dict_allPCA[seg_key]['fitness_score'][1] for seg_key in history_pareto_segments_list if history_segments_dict_allPCA[seg_key]['fitness_score'][1] is not None]

    # Check if both lists are empty or filled with zeros
    if not objective_1_values or not objective_2_values or all(value == 0 for value in objective_1_values + objective_2_values):
        print("All objectives are None or 0, skipping further processes.")
        return  # Exit the function

    non_dominated_segments = {}
    for i1, key1 in enumerate(history_pareto_segments_list):
        if history_segments_dict_allPCA[key1]['fitness_score'] == (None, None):
            continue  # Skip non-evaluable segments

        dominated = False
        for i2, key2 in enumerate(history_pareto_segments_list):
            if i1 != i2 and dominates(history_segments_dict_allPCA[key2]['fitness_score'], history_segments_dict_allPCA[key1]['fitness_score']):
                dominated = True
                break
        if not dominated:
            non_dominated_segments[key1] = history_segments_dict_allPCA[key1]

    pareto_fitness_tuples = [(ft[0], ft[1]) for ft in [history_segments_dict_allPCA[seg_key]['fitness_score'] for seg_key in non_dominated_segments.keys()] if ft[0] is not None and ft[1] is not None]
    pareto_segments_tuples = [(seg_key[2], seg_key[3], history_segments_dict_allPCA[seg_key]['number_of_syn_sample'], history_segments_dict_allPCA[seg_key]['retrained_dots_list']) for seg_key in non_dominated_segments.keys()]

    # Calculate statistics
    worst_fitness = (min(objective_1_values), min(objective_2_values))
    best_fitness = (max(objective_1_values), max(objective_2_values))
    mean_OverallAcc = np.mean(objective_1_values)
    mean_ClassRecall = np.mean(objective_2_values)

    print(f"All Pareto Segments in History, Evaluations: {len(history_pareto_segments_list)}")
    print(f"Best Fitness: {best_fitness}")
    print(f"Worst Fitness: {worst_fitness}")
    print(f"Mean Overall Accuracy: {mean_OverallAcc}")
    print(f"Mean {topic_name} Recall: {mean_ClassRecall}")

    print("Pareto Front Selections:---------------------")
    for i, (key, seg) in enumerate(non_dominated_segments.items()):
        print(f"Segment {i+1}: {seg['coordinates']}, {seg['PCA']}, Num Samples {seg['number_of_syn_sample']}: {seg['retrained_dots_list']} \nFitness: {seg['fitness_score']}")

        classification_df = seg['classification_df']
        # Collect all generation data into a new DataFrame row
        new_PFs_row = {
            "topic_name": topic_name,
            "topic_number": topic_number,
            "PCA_index": used_pca_pairs.index(seg['PCA']),
            "PCA": seg['PCA'],
            "coordinates": seg['coordinates'],
            'fitness_score': seg['fitness_score'],
            "accuracy": seg['fitness_score'][0],
            "topic_recall": seg['fitness_score'][1],
            "retraining_time": seg["retraining_time"],
            "number_of_syn_sample": seg['number_of_syn_sample'],
            "retrained_dots_list": seg['retrained_dots_list'],
            'true_labels': seg['true_labels'],
            'predicted_labels': seg['predicted_labels'],
            "segment_key": key
        }
        # # Transform DataFrame to dict format
        # for idx in classification_df.index:
        #     if idx != 'accuracy' and idx != 'macro avg' and idx != 'weighted avg':
        #         new_PFs_row[idx] = classification_df.loc[idx].dropna().to_dict()

        # Convert the dictionary to a DataFrame for a single row
        new_PFs_row_df = pd.DataFrame([new_PFs_row])
        # Concatenate this new row DataFrame to the existing DataFrame
        fold_pfs_df = pd.concat([fold_pfs_df, new_PFs_row_df], ignore_index=True)
        fold_pfs_df.to_csv(f'{NSW_results_path}/NSWPFs_{NSW_results_name}.csv', index=False)
        fold_pfs_df.to_pickle(f'{NSW_results_path}/NSWPFs_{NSW_results_name}.pkl')
        print('---')
    print('------------------------')

    # Plotting
    plt.figure(figsize=(12, 7))
    plt.scatter(objective_1_values, objective_2_values, c='blue', alpha=0.5, label='Population')
    if pareto_fitness_tuples:
        plt.scatter([ft[0] for ft in pareto_fitness_tuples], [ft[1] for ft in pareto_fitness_tuples], c='red', alpha=0.9, label='Pareto Front')
    
    plt.xlabel('Objective 1: Overall Accuracy')
    plt.ylabel(f'Objective 2: {topic_name} Recall')
    plt.title(f'{topic_number}, Population and Pareto Front for All Pareto Segments in History')
    plt.legend()
    pareto_plots_dir = f"{NSW_results_path}/{PCA_stats_df_name}"
    os.makedirs(pareto_plots_dir, exist_ok=True)
    filename = f"All Pareto Segments in History.png"
    plt.savefig(os.path.join(pareto_plots_dir, filename), dpi=200, bbox_inches='tight', pad_inches=0)
    plt.show()

    recall_key = f"Mean {topic_name} Recall"
    # Collect all generation data into a new DataFrame row
    new_row = {
        "PCA": f"All PCAs",
        "Number of Evaluations": len(history_pareto_segments_list),
        "Best Fitness": best_fitness,
        "Worst Fitness": worst_fitness,
        "Mean Overall Accuracy": mean_OverallAcc,
        recall_key: mean_ClassRecall,
        "Pareto Front Segments": [pareto_segments_tuples, pareto_fitness_tuples]
    }
    # Convert the dictionary to a DataFrame for a single row
    new_row_df = pd.DataFrame([new_row])
    # Concatenate this new row DataFrame to the existing DataFrame
    PCA_stats_df = pd.concat([PCA_stats_df, new_row_df], ignore_index=True)
    PCA_stats_df.to_csv(f'{NSW_results_path}/{PCA_stats_df_name}/All Pareto Segments in History.csv', index=False)
    PCA_stats_df.to_pickle(f'{NSW_results_path}/{PCA_stats_df_name}/All Pareto Segments in History.pkl')


def pca_pareto_observer(history_pareto_segments_list, pca_pair):
    global history_segments_dict_allPCA
    global topic_name
    global topic_number
    global NSW_results_path
    global NSW_results_name
    global PCA_stats_df_name
    global pca_pfs_df
    global used_pca_pairs

    # Filter the list to include only tuples where the (pca1, pca2) matches the input pca_pair
    filtered_list = [item for item in history_pareto_segments_list if item[2] == pca_pair]

    objective_1_values = [history_segments_dict_allPCA[seg_key]['fitness_score'][0] for seg_key in filtered_list if history_segments_dict_allPCA[seg_key]['fitness_score'][0] is not None]
    objective_2_values = [history_segments_dict_allPCA[seg_key]['fitness_score'][1] for seg_key in filtered_list if history_segments_dict_allPCA[seg_key]['fitness_score'][1] is not None]

    # Check if both lists are empty or filled with zeros
    if not objective_1_values or not objective_2_values or all(value == 0 for value in objective_1_values + objective_2_values):
        print("All objectives are None or 0, skipping further processes.")
        return  # Exit the function

    non_dominated_segments = {}
    for i1, key1 in enumerate(filtered_list):
        if history_segments_dict_allPCA[key1]['fitness_score'] == (None, None):
            continue  # Skip non-evaluable segments

        dominated = False
        for i2, key2 in enumerate(filtered_list):
            if i1 != i2 and dominates(history_segments_dict_allPCA[key2]['fitness_score'], history_segments_dict_allPCA[key1]['fitness_score']):
                dominated = True
                break
        if not dominated:
            non_dominated_segments[key1] = history_segments_dict_allPCA[key1]

    pareto_fitness_tuples = [(ft[0], ft[1]) for ft in [history_segments_dict_allPCA[seg_key]['fitness_score'] for seg_key in non_dominated_segments.keys()] if ft[0] is not None and ft[1] is not None]
    # pareto_segments_tuples = [(seg_key[2], seg_key[3], history_segments_dict_allPCA[seg_key]['number_of_syn_sample'], history_segments_dict_allPCA[seg_key]['retrained_dots_list']) for seg_key in non_dominated_segments.keys()]

    # Calculate statistics
    worst_fitness = (min(objective_1_values), min(objective_2_values))
    best_fitness = (max(objective_1_values), max(objective_2_values))
    mean_OverallAcc = np.mean(objective_1_values)
    mean_ClassRecall = np.mean(objective_2_values)

    print(f"All Pareto Segments in {pca_pair}, Evaluations: {len(filtered_list)}")
    print(f"Best Fitness: {best_fitness}")
    print(f"Worst Fitness: {worst_fitness}")
    print(f"Mean Overall Accuracy: {mean_OverallAcc}")
    print(f"Mean {topic_name} Recall: {mean_ClassRecall}")

    print("Pareto Front Selections:---------------------")
    for i, (key, seg) in enumerate(non_dominated_segments.items()):
        print(f"Segment {i+1}: {seg['coordinates']}, {seg['PCA']}, Num Samples {seg['number_of_syn_sample']}: {seg['retrained_dots_list']} \nFitness: {seg['fitness_score']}")
        
        classification_df = seg['classification_df']
        # Collect all generation data into a new DataFrame row
        pca_PFs_row = {
            "topic_name": topic_name,
            "topic_number": topic_number,
            "PCA_index": used_pca_pairs.index(seg['PCA']),
            "PCA": seg['PCA'],
            "coordinates": seg['coordinates'],
            'fitness_score': seg['fitness_score'],
            "accuracy": seg['fitness_score'][0],
            "topic_recall": seg['fitness_score'][1],
            'balanced_fitness_score': (classification_df.loc['accuracy', 'Balanced Accuracy'], classification_df.loc[topic_name, 'Balanced Accuracy']),
            'overall_balanced_accuracy': classification_df.loc['accuracy', 'Balanced Accuracy'],
            'topic_balanced_accuracy': classification_df.loc[topic_name, 'Balanced Accuracy'],
            'balanced_acc_rec_score': (classification_df.loc[topic_name, 'Balanced Accuracy'], classification_df.loc[topic_name, 'recall']),
            'topic_F1': classification_df.loc[topic_name, 'f1-score'],
            'overall_F1': classification_df.loc['accuracy', 'f1-score'],
            'overall_recall': classification_df.loc['accuracy', 'recall'],
            "retraining_time": seg["retraining_time"],
            "number_of_syn_sample": seg['number_of_syn_sample'],
            "retrained_dots_list": seg['retrained_dots_list'],
            'true_labels': seg['true_labels'],
            'predicted_labels': seg['predicted_labels'],
            "segment_key": key
        }
        # # Transform DataFrame to dict format
        # for idx in classification_df.index:
        #     if idx != 'accuracy' and idx != 'macro avg' and idx != 'weighted avg':
        #         pca_PFs_row[idx] = classification_df.loc[idx].dropna().to_dict()
        # Convert the dictionary to a DataFrame for a single row
        pca_PFs_row_df = pd.DataFrame([pca_PFs_row])
        # Concatenate this new row DataFrame to the existing DataFrame
        pca_pfs_df = pd.concat([pca_pfs_df, pca_PFs_row_df], ignore_index=True)
        pca_pfs_df.to_csv(f'{NSW_results_path}/NSWpcaPFs_{NSW_results_name}.csv', index=False)
        pca_pfs_df.to_pickle(f'{NSW_results_path}/NSWpcaPFs_{NSW_results_name}.pkl')
        print('---')
    print('------------------------')

    # Plotting
    plt.figure(figsize=(12, 7))
    plt.scatter(objective_1_values, objective_2_values, c='blue', alpha=0.5, label='Population')
    if pareto_fitness_tuples:
        plt.scatter([ft[0] for ft in pareto_fitness_tuples], [ft[1] for ft in pareto_fitness_tuples], c='red', alpha=0.9, label='Pareto Front')
    
    plt.xlabel('Objective 1: Overall Accuracy')
    plt.ylabel(f'Objective 2: {topic_name} Recall')
    plt.title(f'{topic_number}, Population and Pareto Front for All Pareto Segments in {pca_pair}')
    plt.legend()
    pareto_plots_dir = f"{NSW_results_path}/{PCA_stats_df_name}"
    os.makedirs(pareto_plots_dir, exist_ok=True)
    filename = f"All Pareto Segments in {pca_pair}.png"
    plt.savefig(os.path.join(pareto_plots_dir, filename), dpi=200, bbox_inches='tight', pad_inches=0)
    plt.show()

In [4]:
def segment_retraining(data_syn, individual_Segment_dict, segment_key):
    global dots_mode
    global history_segments_dict
    global X_train_r
    global Y_train_r
    global X_test_re
    global Y_test_re
    global catboost_params
    global sum_GPU_seconds
    global total_gpu_seconds
    global GPU_limit
    global X_test_re_Test
    global Y_test_re_Test
    global test_fold_results_df
    global bch_m0
    global NSW_results_path
    global used_pca_pairs
    global topic_name
    global topic_number
    global NSW_results_name

    CPU_monitor_memory_usage()
    monitor_gpu_memory()

    if dots_mode == "False":
        syn_original_list = individual_Segment_dict["red_dots_list"]
    elif dots_mode == "Both":
        syn_original_list = individual_Segment_dict["red_dots_list"] + individual_Segment_dict["blue_dots_list"]

    if len(syn_original_list) == 0:
        individual_Segment_dict['model'] = None
        individual_Segment_dict['true_labels'] = None
        individual_Segment_dict['predicted_labels'] = None
        individual_Segment_dict['classification_df'] = None
        individual_Segment_dict['fitness_score'] = (None, None)
        individual_Segment_dict['number_of_syn_sample'] = None
        individual_Segment_dict['retraining_time'] = None
        individual_Segment_dict['retrained_dots_list'] = []
        return individual_Segment_dict

    for previous_segment_key, previous_Segment_dict in history_segments_dict.items():
        if syn_original_list == previous_Segment_dict['retrained_dots_list']:
            individual_Segment_dict['model'] = previous_Segment_dict['model']
            individual_Segment_dict['true_labels'] = previous_Segment_dict['true_labels']
            individual_Segment_dict['predicted_labels'] = previous_Segment_dict['predicted_labels']
            individual_Segment_dict['classification_df'] = previous_Segment_dict['classification_df']
            individual_Segment_dict['fitness_score'] = previous_Segment_dict['fitness_score']
            individual_Segment_dict['number_of_syn_sample'] = previous_Segment_dict['number_of_syn_sample']
            individual_Segment_dict['retraining_time'] = previous_Segment_dict['retraining_time']
            individual_Segment_dict['retrained_dots_list'] = syn_original_list
            return individual_Segment_dict
    
    filtered_syn_df = data_syn[data_syn['index_meta'].isin(syn_original_list)]

    X_train_re = pd.concat([X_train_r, filtered_syn_df.drop(columns=['topic_name'])])
    Y_train_re = pd.concat([Y_train_r, filtered_syn_df['topic_name']])

    train_pool_re = Pool(
        X_train_re[["text", "area_TEIS"]],
        Y_train_re,
        text_features=["text"],
        cat_features=["area_TEIS"]
    )
    valid_pool_re = Pool(
        X_test_re[["text", "area_TEIS"]],
        Y_test_re,
        text_features=["text"],
        cat_features=["area_TEIS"]
    )

    catboost_params = catboost_params
            
    # Model Training
    model_re = CatBoostClassifier(**catboost_params)
    start_time = time.time()  # Start timing
    model_re.fit(train_pool_re, eval_set=valid_pool_re)
    training_time = time.time() - start_time  # End timing

    sum_GPU_seconds += training_time
    if sum_GPU_seconds >= total_gpu_seconds:
        GPU_limit = True

    # Save the retrain performances
    predictions = model_re.predict(X_test_re[["text", "area_TEIS"]])
    accuracy = accuracy_score(Y_test_re, predictions)
    report = classification_report(Y_test_re, predictions, digits=3, output_dict=True)
    classification_df = classification_report_to_df(report, Y_test_re, predictions)
    
    fitness_score = (accuracy, classification_df.loc[topic_name, 'recall'])
            
    # Save the trained model, classification_df, and fitness_score
    individual_Segment_dict['model'] = model_re
    individual_Segment_dict['true_labels'] = []  # Convert to list if Y_test_re is a pandas Series or numpy array
    individual_Segment_dict['predicted_labels'] = []  # Convert to list for consistency
    individual_Segment_dict['classification_df'] = classification_df
    individual_Segment_dict['fitness_score'] = fitness_score
    individual_Segment_dict['number_of_syn_sample'] = len(filtered_syn_df)
    individual_Segment_dict['retraining_time'] = training_time  # Save the training time
    individual_Segment_dict['retrained_dots_list'] = syn_original_list

    """Testing results below"""

    test_predictions = model_re.predict(X_test_re_Test[["text", "area_TEIS"]])
    test_accuracy = accuracy_score(Y_test_re_Test, test_predictions)
    test_report = classification_report(Y_test_re_Test, test_predictions, digits=3, output_dict=True)
    test_classification_df = classification_report_to_df(test_report, Y_test_re_Test, test_predictions)

    test_new_row_index = len(test_fold_results_df)
    test_class_DF_path = f'{NSW_results_path}/test_Class_DF'
    os.makedirs(test_class_DF_path, exist_ok=True)
    test_classification_df.to_csv(f'{test_class_DF_path}/test_{topic_number}_NSW_{test_new_row_index}_AllEval_ClassDF.csv', index=True)
    test_classification_df.to_pickle(f'{test_class_DF_path}/test_{topic_number}_NSW_{test_new_row_index}_AllEval_ClassDF.pkl')

    # Collect all generation data into a new DataFrame row
    test_new_ParamCV_row = {
        "topic_name": topic_name,
        "topic_number": topic_number,
        "PCA_index": used_pca_pairs.index(individual_Segment_dict['PCA']),
        "PCA": individual_Segment_dict['PCA'],
        "coordinates": individual_Segment_dict['coordinates'],
        'fitness_score': individual_Segment_dict['fitness_score'],
        "accuracy": individual_Segment_dict['fitness_score'][0],
        "topic_recall": individual_Segment_dict['fitness_score'][1],
        'balanced_fitness_score': (test_classification_df.loc['accuracy', 'Balanced Accuracy'], test_classification_df.loc[topic_name, 'Balanced Accuracy']),
        'overall_balanced_accuracy': test_classification_df.loc['accuracy', 'Balanced Accuracy'],
        'topic_balanced_accuracy': test_classification_df.loc[topic_name, 'Balanced Accuracy'],
        'balanced_acc_rec_score': (test_classification_df.loc[topic_name, 'Balanced Accuracy'], test_classification_df.loc[topic_name, 'recall']),
        'topic_F1': test_classification_df.loc[topic_name, 'f1-score'],
        'overall_F1': test_classification_df.loc['accuracy', 'f1-score'],
        'overall_recall': test_classification_df.loc['accuracy', 'recall'],
        "retraining_time": individual_Segment_dict["retraining_time"],
        "number_of_syn_sample": individual_Segment_dict['number_of_syn_sample'],
        "retrained_dots_list": individual_Segment_dict['retrained_dots_list'],
        'true_labels': individual_Segment_dict['true_labels'],
        'predicted_labels': individual_Segment_dict['predicted_labels'],
        "segment_key": (topic_name, topic_number, individual_Segment_dict['PCA'], individual_Segment_dict['coordinates']),
        'classDF_path': f'{test_class_DF_path}/test_{topic_number}_NSW_{test_new_row_index}_AllEval_ClassDF.csv',
        'T13_TBA_Imp': test_classification_df.loc['Humanitarian aid for Ukraine.', 'Balanced Accuracy'] - bch_m0.loc['Humanitarian aid for Ukraine.', 'Balanced Accuracy'],
        'T13_TR_Imp': classification_df.loc['Humanitarian aid for Ukraine.', 'recall'] - bch_m0.loc['Humanitarian aid for Ukraine.', 'recall']
    }
    # # Transform DataFrame to dict format
    # for idx in classification_df.index:
    #     if idx != 'accuracy' and idx != 'macro avg' and idx != 'weighted avg':
    #         new_ParamCV_row[idx] = classification_df.loc[idx].dropna().to_dict()


    # Convert the dictionary to a DataFrame for a single row
    test_new_ParamCV_row_df = pd.DataFrame([test_new_ParamCV_row])
    # Concatenate this new row DataFrame to the existing DataFrame
    test_fold_results_df = pd.concat([test_fold_results_df, test_new_ParamCV_row_df], ignore_index=True)
    test_fold_results_df.to_csv(f'{NSW_results_path}/test_NSWAllSegs_{NSW_results_name}.csv', index=False)
    test_fold_results_df.to_pickle(f'{NSW_results_path}/test_NSWAllSegs_{NSW_results_name}.pkl')

    

    return individual_Segment_dict


def sliding_window_get_dots(history_segments_dict, tr_df, data_syn, topic_name, topic_number, area_size_x, area_size_y, level_x_start, level_y_start, pca_pairs = [("PCA_0", "PCA_1")], num_segments=16, window_size=1):
    global sw_level
    global NSW_results_path
    global all_segments_dict_name
    global GPU_limit

    for pca1, pca2 in pca_pairs:
        if GPU_limit == True:
            return history_segments_dict
        # Separate data points by their classification status
        grey_mask = tr_df['topic_name'] != topic_name
        blue_mask = (tr_df['topic_name'] == topic_name) & (tr_df['pred_topic_name'] == topic_name)
        red_mask = (tr_df['topic_name'] == topic_name) & (tr_df['pred_topic_name'] != topic_name)

        if area_size_x == "FullSize":
            segment_size_x = (tr_df[pca1].max() - tr_df[pca1].min()) / num_segments
        else:
            segment_size_x = (area_size_x[1] - area_size_x[0]) / num_segments
        if area_size_y == "FullSize":
            segment_size_y = (tr_df[pca2].max() - tr_df[pca2].min()) / num_segments
        else:
            segment_size_y = (area_size_y[1] - area_size_y[0]) / num_segments

        # Sliding window through the plot area
        for i in range(num_segments - window_size + 1):
            for j in range(num_segments - window_size + 1):
                if GPU_limit == True:
                    return history_segments_dict
                individual_Segment_dict = {}
                
                if level_x_start == "minimum":
                    x_start = tr_df[pca1].min() + i * segment_size_x
                else:
                    x_start = level_x_start + i * segment_size_x
                if level_y_start == "minimum":
                    y_start = tr_df[pca2].min() + j * segment_size_y
                else:
                    y_start = level_y_start + j * segment_size_y
                
                x_end = x_start + window_size * segment_size_x
                y_end = y_start + window_size * segment_size_y

                coordinates = f"{x_start}-{x_end}, {y_start}-{y_end}"
                segment_key = (topic_name, topic_number, (pca1, pca2), coordinates)

                # Check if a segment with these specific details already exists
                if any(key == segment_key for key in history_segments_dict.keys()):
                    continue  # Skip this iteration if a match is found

                segment_mask = (tr_df[pca1] >= x_start) & (tr_df[pca1] <= x_end) & (tr_df[pca2] >= y_start) & (tr_df[pca2] <= y_end)
                segment_red_mask = segment_mask & red_mask
                segment_blue_mask = segment_mask & blue_mask
                segment_grey_mask =  segment_mask & grey_mask
                
                individual_Segment_dict["coordinates"] = coordinates
                individual_Segment_dict["PCA"] = (pca1, pca2)
                individual_Segment_dict["blue_dots_list"] = tr_df.loc[segment_blue_mask, 'index_meta'].tolist()
                individual_Segment_dict["red_dots_list"] = tr_df.loc[segment_red_mask, 'index_meta'].tolist()
                individual_Segment_dict["grey_dots_list"] = tr_df.loc[segment_grey_mask, 'index_meta'].tolist()

                individual_Segment_dict = segment_retraining(data_syn, individual_Segment_dict, segment_key)

                if individual_Segment_dict['fitness_score'] != (None, None):
                    history_segments_dict[segment_key] = individual_Segment_dict
                    print(individual_Segment_dict['fitness_score'])
                    
                    os.makedirs(f'{NSW_results_path}/{all_segments_dict_name}', exist_ok=True)
                    with open(f'{NSW_results_path}/{all_segments_dict_name}/{pca1}{pca2}.pkl', 'wb') as file:
                        pickle.dump(history_segments_dict, file)
    
        print(f'Finished hierarchical sliding window for {topic_name}_{pca1}_{pca2}.')
    return history_segments_dict

In [5]:
def dominates(score1, score2):
    """
    Determines if one score dominates another.
    A score1 dominates score2 if it is better in all the objectives or equal in some and better in at least one.
    """
    return (score1[0] > score2[0] and score1[1] >= score2[1]) or (score1[0] >= score2[0] and score1[1] > score2[1])

def find_pareto_front(df):
    """
    Marks rows as 'Yes' if they are on the Pareto front, 'No' otherwise.
    """
    df = df.copy()  # Copy DataFrame to avoid modifying the original
    df['Pareto'] = 'No'  # Initialize the Pareto column with 'No'
    
    scores = df['balanced_acc_rec_score'].tolist()
    is_pareto = np.ones(len(scores), dtype=bool)  # Initialize all as True
    
    for i1 in range(len(scores)):
        for i2 in range(len(scores)):
            if i1 != i2 and dominates(scores[i2], scores[i1]):
                is_pareto[i1] = False
                break

    # Update the 'Pareto' column based on the Pareto front
    df.loc[is_pareto, 'Pareto'] = 'Yes'
    
    return df

def find_best_values(df):
    # Identify the maximum values for each specified column
    max_values = {
        'accuracy': df['accuracy'].max(),
        'topic_recall': df['topic_recall'].max(),
        'overall_balanced_accuracy': df['overall_balanced_accuracy'].max(),
        'topic_balanced_accuracy': df['topic_balanced_accuracy'].max(),
        'topic_F1': df['topic_F1'].max(),
        'overall_F1': df['overall_F1'].max(),
        'overall_recall': df['overall_recall'].max()
    }
    
    # Function to apply to each row to determine the best columns
    def check_best(row):
        return [col for col, max_val in max_values.items() if row[col] == max_val]

    # Apply the function to each row
    df['best'] = df.apply(check_best, axis=1)
    
    return df

def post_process(df, bch_class_df):
    global topic_number
    topic_name = topic_dict[topic_number]

    bch_topic_recall = bch_class_df.loc[topic_name, 'recall']
    bch_topic_balanced_accuracy = bch_class_df.loc[topic_name, 'Balanced Accuracy']
    bch_overall_balanced_accuracy = bch_class_df.loc['accuracy', 'Balanced Accuracy']
    bch_overall_F1_score = bch_class_df.loc['accuracy', 'f1-score']

    # Calculate improvements
    df['imp_topic_recall'] = df['topic_recall'] - bch_topic_recall
    df['imp_topic_balanced_accuracy'] = df['topic_balanced_accuracy'] - bch_topic_balanced_accuracy
    df['imp_overall_balanced_accuracy'] = df['overall_balanced_accuracy'] - bch_overall_balanced_accuracy
    df['imp_overall_F1'] = df['overall_F1'] - bch_overall_F1_score

    # Calculate cumulative retraining_time
    df['cumulative_time'] = df['retraining_time'].cumsum()

    # Calculate max and average improvements
    df['max_topic_recall_imp'] = df[['imp_topic_recall']].max(axis=1).cummax()
    df['average_topic_recall_imp'] = df[['imp_topic_recall']].mean(axis=1).expanding().mean()

    df['max_topic_balanced_acc_imp'] = df[['imp_topic_balanced_accuracy']].max(axis=1).cummax()
    df['average_topic_balanced_acc_imp'] = df[['imp_topic_balanced_accuracy']].mean(axis=1).expanding().mean()

    df['max_overall_balanced_acc_imp'] = df[['imp_overall_balanced_accuracy']].max(axis=1).cummax()
    df['average_overall_balanced_acc_imp'] = df[['imp_overall_balanced_accuracy']].mean(axis=1).expanding().mean()

    df['max_overall_F1_improvement'] = df[['imp_overall_F1']].max(axis=1).cummax()
    df['average_overall_F1_improvement'] = df[['imp_overall_F1']].mean(axis=1).expanding().mean()
    return df

In [6]:
def bch_classification_report_to_df(report, y_true, y_pred):
    global bch_class_df
    global topic_dict
    df = pd.DataFrame(report).transpose()

    # Calculate the confusion matrix
    labels = df.index[:-3]  # Exclude 'accuracy', 'macro avg', 'weighted avg'
    # Calculate the confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    # Extracting TP, FP, TN, FN for each class
    TP = cm.diagonal()
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (FP + FN + TP)

    sens = sum(TP) / (sum(TP)+sum(FN))
    spec = sum(TN) / (sum(TN)+sum(FP))
    
    # Calculate Sensitivity (same as recall)
    df['Sensitivity'] = df['recall']
    
    # Calculate Specificity
    tn = cm.sum() - (cm.sum(axis=0) + cm.sum(axis=1) - np.diag(cm))
    fp = cm.sum(axis=0) - np.diag(cm)
    specificity = tn / (tn + fp)
    
    # Assign computed specificity to dataframe except for the last three rows
    df.loc[df.index[:-3], 'Specificity'] = specificity
    
    # Handling special cases
    # Set 'accuracy' row sensitivity and specificity to the accuracy value
    accuracy = df.loc['accuracy', 'precision']  # assuming 'precision' contains the accuracy
    df.loc['accuracy', ['Sensitivity', 'Specificity']] = sens, spec
    
    # Calculate 'macro avg' and 'weighted avg' for sensitivity and specificity
    df.loc['macro avg', 'Sensitivity'] = df.iloc[:-3]['Sensitivity'].mean()
    df.loc['weighted avg', 'Sensitivity'] = np.average(df.iloc[:-3]['Sensitivity'], weights=df.iloc[:-3]['support'])
    
    df.loc['macro avg', 'Specificity'] = df.iloc[:-3]['Specificity'].mean()
    df.loc['weighted avg', 'Specificity'] = np.average(df.iloc[:-3]['Specificity'], weights=df.iloc[:-3]['support'])

    # Calculate Balanced Accuracy for each row, including special averages
    df['Balanced Accuracy'] = (df['Sensitivity'] + df['Specificity']) / 2
    
    return df

def train_bch(X_train_re, X_test_re, Y_train_re, Y_test_re, catboost_params, itr0_path):
    global X_test_re_Test
    global Y_test_re_Test
    CPU_monitor_memory_usage()
    monitor_gpu_memory()
    bch_dict = {}

    train_pool_re = Pool(
        X_train_re[["text", "area_TEIS"]],
        Y_train_re,
        text_features=["text"],
        cat_features=["area_TEIS"]
    )
    valid_pool_re = Pool(
        X_test_re[["text", "area_TEIS"]],
        Y_test_re,
        text_features=["text"],
        cat_features=["area_TEIS"]
    )

    # Model Training
    model_re = CatBoostClassifier(**catboost_params)
    start_time = time.time()  # Start timing
    model_re.fit(train_pool_re, eval_set=valid_pool_re)
    training_time = time.time() - start_time  # End timing

    # Save the retrain performances
    val_predictions = model_re.predict(X_test_re[["text", "area_TEIS"]])
    val_accuracy = accuracy_score(Y_test_re, val_predictions)
    val_report = classification_report(Y_test_re, val_predictions, digits=3, output_dict=True)
    print(val_accuracy)
    # print(report)
    val_classification_df = bch_classification_report_to_df(val_report, Y_test_re, val_predictions)
    # print(classification_df)
    val_classification_df.to_pickle(f"{itr0_path}/Validation_Benchmark_M0_Classdf_0.pkl")
    val_classification_df.to_csv(f"{itr0_path}/Validation_Benchmark_M0_Classdf_0.csv", index=True)

    # Save the retrain performances
    predictions = model_re.predict(X_test_re_Test[["text", "area_TEIS"]])
    accuracy = accuracy_score(Y_test_re_Test, predictions)
    report = classification_report(Y_test_re_Test, predictions, digits=3, output_dict=True)
    print(accuracy)
    # print(report)
    classification_df = bch_classification_report_to_df(report, Y_test_re_Test, predictions)
    # print(classification_df)

    classification_df.to_pickle(f"{itr0_path}/Benchmark_M0_Classdf_0.pkl")
    classification_df.to_csv(f"{itr0_path}/Benchmark_M0_Classdf_0.csv", index=True)

    bch_dict['model'] = model_re
    bch_dict['classification_df'] = classification_df
    bch_dict['accuracy'] = accuracy
    bch_dict['retraining_time'] = training_time

    return bch_dict

In [ ]:
if __name__ == "__main__":
    run = 1
    rand = 10
    
    run_path = f"D:/Step_2_Pathway/Paper_GPU1h_Improve_T13_TBA/C_Run_{run}"
    itr0_path = f"{run_path}/Iteration_0"
    os.makedirs(itr0_path, exist_ok=True)

    # Load Data
    data = pd.read_csv(f'D:/AutoGeTS/Data/tickets_topics.csv',lineterminator='\n')
    data_topic = data.dropna().reset_index()
    data_topic = data_topic.rename(columns={'index': 'index_meta'})

    X_train_r_both, X_test_re_Test, Y_train_r_both, Y_test_re_Test = train_test_split(data_topic, data_topic.topic_name, test_size = 0.2,random_state = 42)
        
    # Further split the training set to create a validation set
    X_train_r, X_test_re, Y_train_r, Y_test_re = train_test_split(
        X_train_r_both, 
        Y_train_r_both, 
        test_size=0.2,  # 20% of the initial training set, which is 16% of the original data
        random_state=rand
    )

    catboost_params = {'iterations': 300, 'learning_rate': 0.2, 'depth': 8, 'l2_leaf_reg': 1, 
                        'bagging_temperature': 1, 'random_strength': 1, 'border_count': 254, 
                        'eval_metric': 'TotalF1', 'task_type': 'GPU', 'early_stopping_rounds': 20, 'use_best_model': True, 'verbose': 0, 'random_seed': rand}

    topic_dict = {"T1": "IT support and assistance.","T2": "Account activation and access issues.","T3": "Password and device security.",
                    "T4": "Printer issues and troubleshooting.","T5": "HP Dock connectivity issues.","T6": "Employee documentation and errors.",
                    "T7": "\"Access and login issues\"","T8": "Opening and managing files/devices.","T9": "Mobile email and VPN setup.",
                    "T10": "IT support and communication.","T11": "Error handling in RPG programming.","T12": "Email security and attachments.",
                    "T13": "Humanitarian aid for Ukraine.","T14": "Internet connectivity issues in offices.","T15": "Improving integration with Infojobs."}

    bch_dict = train_bch(X_train_r, X_test_re, Y_train_r, Y_test_re, catboost_params, itr0_path)
    for iteration in [1]:
        if iteration == 1:
            topic_number = "T5"
        # for topic_number in ["T10"]: # "T3", "T6", "T7", "T8", "T9", "T12"
        # iteration = 3
        prev_itr = iteration - 1

        gpu_hours = 1

        NSW_results_path = f"D:/Step_2_Pathway/Paper_GPU1h_Improve_T13_TBA/C_Run_{run}/Iteration_{iteration}/{topic_number}_GPU{gpu_hours}h_SSW"
        os.makedirs(NSW_results_path, exist_ok=True)
        
        """Results region"""
        fold_results_df = pd.DataFrame() # pd.read_pickle("D:/AutoGeTS/Topic_Experiments/Naive-SW_Results/NSWAllSegs_NSW_T13_ModeBoth_as=T,o,p,i,c,M,i,n,M,a,x_ns=1,6_ws=4_PCA_0PCA_1_PCA_18PCA_19.pkl") # pd.DataFrame() # pd.read_pickle("D:/AutoGeTS/Topic_Experiments/Hierarchical-SW_Results/HSWAllSegs_HSW_T13_ModeBoth_as=T,o,p,i,c,M,i,n,M,a,x_ns=8,4,2_ws=h,a,l,f_PCA_0PCA_1_PCA_18PCA_19.pkl") # pd.DataFrame()  # DataFrame to collect aggregated results
        fold_pfs_df = pd.DataFrame()  # DataFrame to collect details from each fold
        pca_pfs_df = pd.DataFrame() # pd.read_pickle("D:/AutoGeTS/Topic_Experiments/Naive-SW_Results/NSWpcaPFs_NSW_T13_ModeBoth_as=T,o,p,i,c,M,i,n,M,a,x_ns=1,6_ws=4_PCA_0PCA_1_PCA_18PCA_19.pkl") # pd.DataFrame() 
        test_fold_results_df = pd.DataFrame()

        history_segments_dict_allPCA = {}
        history_pareto_segments_list = []
        PCA_stats_df_allPCA = pd.DataFrame(columns=["PCA", "Number of Evaluations", "Best Fitness", "Worst Fitness", "Mean Overall Accuracy", "Mean Topic Recall", "Pareto Front Segments"])

        if iteration > 1:
            bch_class_df = pd.read_pickle(f"D:/Step_2_Pathway/Paper_GPU1h_Improve_T13_TBA/C_Run_{run}/Iteration_{prev_itr}/Bch_Itr_{prev_itr}.pkl")
            bch_filtered_columns = [col for col in bch_class_df.columns if not col.startswith("Diff") and col != "Accuracy"]
            bch_class_df = bch_class_df[bch_filtered_columns]
        else:
            bch_class_df = pd.read_pickle(f"D:/Step_2_Pathway/Paper_GPU1h_Improve_T13_TBA/C_Run_{run}/Iteration_0/Benchmark_M0_Classdf_0.pkl")

        bch_m0 = pd.read_pickle(f"D:/Step_2_Pathway/Paper_GPU1h_Improve_T13_TBA/C_Run_{run}/Iteration_0/Benchmark_M0_Classdf_0.pkl")
        
        if iteration > 1:
            prev_itr_X_train_re = pd.read_pickle(f"D:/Step_2_Pathway/Paper_GPU1h_Improve_T13_TBA/C_Run_{run}/Iteration_{prev_itr}/X_train_re_itr_{prev_itr}.pkl")
            X_train_r = prev_itr_X_train_re
            prev_itr_Y_train_re = pd.read_pickle(f"D:/Step_2_Pathway/Paper_GPU1h_Improve_T13_TBA/C_Run_{run}/Iteration_{prev_itr}/Y_train_re_itr_{prev_itr}.pkl")
            Y_train_r = prev_itr_Y_train_re

        index_meta_values = X_train_r['index_meta'].unique()
        train_PCA_YZ_df = pd.read_pickle("D:/AutoGeTS/Data/Train_PCA_YZ_withPred_0.pkl")
        train_PCA_YZ_df = train_PCA_YZ_df[train_PCA_YZ_df['index_meta'].isin(index_meta_values)]
        # train_PCA_YZ_df = train_PCA_YZ_df.rename(columns={'index': 'index_meta'})
        pca_columns = [col for col in train_PCA_YZ_df.columns if 'PCA_' in col]
        pca_pairs = list(itertools.combinations(pca_columns, 2))

        topic_dict = {"T1": "IT support and assistance.","T2": "Account activation and access issues.","T3": "Password and device security.",
                    "T4": "Printer issues and troubleshooting.","T5": "HP Dock connectivity issues.","T6": "Employee documentation and errors.",
                    "T7": "\"Access and login issues\"","T8": "Opening and managing files/devices.","T9": "Mobile email and VPN setup.",
                    "T10": "IT support and communication.","T11": "Error handling in RPG programming.","T12": "Email security and attachments.",
                    "T13": "Humanitarian aid for Ukraine.","T14": "Internet connectivity issues in offices.","T15": "Improving integration with Infojobs."}

        """Parameters and Input Section ----------------"""
        # topic_number = "T13"
        syn_number = 1

        dots_mode = "Both"

        total_gpu_seconds = gpu_hours * 60 * 60
        
        # Added synthetic data path
        if topic_number in ["T1", "T2"]:
            data_syn_raw = pd.read_pickle(f'D:/AutoGeTS/Synthetic_Data/{topic_number}-synthesis-{syn_number}.pkl')
        else:
            data_syn_raw = pd.read_csv(f'D:/AutoGeTS/Synthetic_Data/{topic_number}-synthesis-{syn_number}.csv',lineterminator='\n')
        data_syn = data_syn_raw[["index_meta", "text", "area_TEIS", 'topic_name', "sample"]].dropna()
    
        used_pca_pairs = pca_pairs # pca_pairs, [("PCA_0", "PCA_5")]

        NSW_params = {'area_size': "TopicMinMax",
                    'num_segments': '16',
                    'window_size': '4'
                    }

        NSW_results_name = f"NSW_{topic_number}_Mode{dots_mode}_{dict_to_foldername(NSW_params)}_{''.join(item for item in used_pca_pairs[0])}_{''.join(item for item in used_pca_pairs[-1])}"

        """------------------------------------------------"""
        topic_name = topic_dict[topic_number]
        clean_topic_name = clean_folder_name(topic_name)

        sum_GPU_seconds = 0
        GPU_limit = False

        for pca_i, pca_pair in enumerate(used_pca_pairs):
            # if pca_i <= 47:
            #     continue
            print(f"Pair {pca_i}", pca_pair)
            CPU_monitor_memory_usage()
            monitor_gpu_memory()

            current_used_pca_pairs = [pca_pair]

            if NSW_params['area_size'] == "FullSize":
                area_size_x = "FullSize"
                area_size_y = "FullSize"
                x_start = "minimum"
                y_start = "minimum"
            elif NSW_params['area_size'] == "TopicMinMax":
                filtered_df = train_PCA_YZ_df[train_PCA_YZ_df['topic_name'] == topic_name]
                area_size_x =(filtered_df[pca_pair[0]].min()*1.01, filtered_df[pca_pair[0]].max()*1.01)
                area_size_y =(filtered_df[pca_pair[1]].min()*1.01, filtered_df[pca_pair[1]].max()*1.01)
                x_start = area_size_x[0] 
                y_start = area_size_y[0]
            else:
                area_size_x = NSW_params['area_size']
                area_size_y = NSW_params['area_size']
                x_start = area_size_x[0]
                y_start = area_size_y[0]
            num_segments = int(NSW_params['num_segments'])
            window_size = int(NSW_params['window_size'])

            history_segments_dict = {}
            PCA_stats_df = pd.DataFrame(columns=["PCA", "Number of Evaluations", "Best Fitness", "Worst Fitness", "Mean Overall Accuracy", "Mean Topic Recall", "Pareto Front Segments"])
            all_segments_dict_name = f"NSW-Retrain-Dict_{topic_number}_Mode{dots_mode}_{''.join(item for item in used_pca_pairs[0])}_NS{num_segments}_WS{window_size}"
            PCA_stats_df_name = f"NSW-PCA-Stats_{topic_number}_Mode{dots_mode}_{''.join(item for item in used_pca_pairs[0])}_NS{num_segments}_WS{window_size}"

            all_segments_dict = sliding_window_get_dots(history_segments_dict, train_PCA_YZ_df, data_syn, topic_name, topic_number, area_size_x, area_size_y, x_start, y_start, pca_pairs = current_used_pca_pairs, num_segments=num_segments, window_size=window_size)
            history_segments_dict_allPCA.update(all_segments_dict)
            # with open(f'{NSW_results_path}/{all_segments_dict_name}_AllPCADictsList.pkl', 'wb') as file:
            #     pickle.dump(history_segments_dict_allPCA, file)
            
            observer_function(all_segments_dict, pca_pair)
            PCA_stats_df_allPCA = pd.concat([PCA_stats_df_allPCA, PCA_stats_df], axis=0, ignore_index=True)
            PCA_stats_df_allPCA.to_csv(f'{NSW_results_path}/{PCA_stats_df_name}.csv', index=False)
            PCA_stats_df_allPCA.to_pickle(f'{NSW_results_path}/{PCA_stats_df_name}.pkl')

            pca_pareto_observer(history_pareto_segments_list, pca_pair)
            if GPU_limit == True:
                fold_results_df  = find_pareto_front(fold_results_df)
                fold_results_df = find_best_values(fold_results_df)
                fold_results_df = post_process(fold_results_df, bch_class_df)
                fold_results_df.to_csv(f'{NSW_results_path}/NSWAllSegs_{NSW_results_name}.csv', index=False)
                fold_results_df.to_pickle(f'{NSW_results_path}/NSWAllSegs_{NSW_results_name}.pkl')
                # break
                test_fold_results_df  = find_pareto_front(test_fold_results_df)
                test_fold_results_df = find_best_values(test_fold_results_df)
                test_fold_results_df = post_process(test_fold_results_df, bch_class_df)
                test_fold_results_df.to_csv(f'{NSW_results_path}/test_NSWAllSegs_{NSW_results_name}.csv', index=True)
                test_fold_results_df.to_pickle(f'{NSW_results_path}/test_NSWAllSegs_{NSW_results_name}.pkl')
                break
        
        """Extract best model and append synthetics"""
        # Find the index of the row with the largest value in the 'max_overall_balanced_acc_imp' column
        index_of_max_imp = test_fold_results_df['imp_overall_balanced_accuracy'].idxmax()
        print(index_of_max_imp)

        # Retrieve the row corresponding to this index
        row_with_largest_value = test_fold_results_df.loc[index_of_max_imp]

        filtered_syn_df = data_syn[data_syn['index_meta'].isin(row_with_largest_value['retrained_dots_list'])]

        X_train_re = pd.concat([X_train_r, filtered_syn_df.drop(columns=['topic_name'])])
        Y_train_re = pd.concat([Y_train_r, filtered_syn_df['topic_name']])

        train_pool_re = Pool(
            X_train_re[["text", "area_TEIS"]],
            Y_train_re,
            text_features=["text"],
            cat_features=["area_TEIS"]
        )
        valid_pool_re = Pool(
            X_test_re[["text", "area_TEIS"]],
            Y_test_re,
            text_features=["text"],
            cat_features=["area_TEIS"]
        )

        catboost_params = catboost_params
                    
        # Model Training
        model_re = CatBoostClassifier(**catboost_params)
        # start_time = time.time()  # Start timing
        model_re.fit(train_pool_re, eval_set=valid_pool_re)
        # training_time = time.time() - start_time  # End timing

        # Save the retrain performances
        predictions = model_re.predict(X_test_re_Test[["text", "area_TEIS"]])
        accuracy = accuracy_score(Y_test_re_Test, predictions)
        report = classification_report(Y_test_re_Test, predictions, digits=6, output_dict=True)
        classification_df = classification_report_to_df(report, Y_test_re_Test, predictions)

        print(classification_df)

        iteration_repo = f"D:/Step_2_Pathway/Paper_GPU1h_Improve_T13_TBA/C_Run_{run}/Iteration_{iteration}/"
        os.makedirs(iteration_repo, exist_ok=True)
        if classification_df.loc[topic_dict["T13"], 'Diff Balanced Accuracy'] >= 0:
            classification_df.to_csv(f"{iteration_repo}/Bch_Itr_{iteration}.csv", index=True)
            classification_df.to_pickle(f"{iteration_repo}/Bch_Itr_{iteration}.pkl")

            X_train_re.to_pickle(f"{iteration_repo}/X_train_re_itr_{iteration}.pkl")
            Y_train_re.to_pickle(f"{iteration_repo}/Y_train_re_itr_{iteration}.pkl")
        else:
            iteration_noimprove_repo = f"D:/Step_2_Pathway/Paper_GPU1h_Improve_T13_TBA/C_Run_{run}/Iteration_{iteration}/Itr_No_Improve"
            os.makedirs(iteration_noimprove_repo, exist_ok=True)
            classification_df.to_csv(f"{iteration_noimprove_repo}/Bch_Itr_{iteration}.csv", index=True)
            classification_df.to_pickle(f"{iteration_noimprove_repo}/Bch_Itr_{iteration}.pkl")

            X_train_re.to_pickle(f"{iteration_noimprove_repo}/X_train_re_itr_{iteration}.pkl")
            Y_train_re.to_pickle(f"{iteration_noimprove_repo}/Y_train_re_itr_{iteration}.pkl")

            bch_class_df.to_csv(f"{iteration_repo}/Bch_Itr_{iteration}.csv", index=True)
            bch_class_df.to_pickle(f"{iteration_repo}/Bch_Itr_{iteration}.pkl")

            prev_itr_X_train_re.to_pickle(f"{iteration_repo}/X_train_re_itr_{iteration}.pkl")
            prev_itr_Y_train_re.to_pickle(f"{iteration_repo}/Y_train_re_itr_{iteration}.pkl")
        # PCA_stats_df_allPCA = pd.concat([PCA_stats_df_allPCA, PCA_stats_df], axis=0, ignore_index=True)
        # PCA_stats_df_allPCA.to_csv(f'{NSW_results_path}/{PCA_stats_df_name}.csv', index=False)
        # PCA_stats_df_allPCA.to_pickle(f'{NSW_results_path}/{PCA_stats_df_name}.pkl')